In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sos4hjb.polynomials import Variable, MonomialVector, ChebyshevVector, Polynomial
from sos4hjb.optimization.drake import SosProgram
from sos4hjb.plot_utils import level_plot

# System dynamics

In [ ]:
# State with limits.
x = Variable.multivariate('x', 3)
xlim = np.array([1, 1, 1])
xobj = xlim / 3
x_m = [MonomialVector.make_polynomial(xi) for xi in x]
xlim_m = [MonomialVector.make_polynomial(xlimi) for xlimi in xlim]
X = [(xi + xlimi) * (xlimi - xi) for xi, xlimi in zip(x_m, xlim_m)]

# Input with limits.
u = Variable.multivariate('u', 2)
ulim = np.array([1, 1])
u_m = [MonomialVector.make_polynomial(ui) for ui in u]
ulim_m = [MonomialVector.make_polynomial(ulimi) for ulimi in ulim]
U = [(ui + ulimi) * (ulimi - ui) for ui, ulimi in zip(u_m, ulim_m)]

# Dynamics.
f = [
    u_m[0],
    u_m[1],
    u_m[0] * x_m[1] - u_m[1] * x_m[0]
]

# Running cost.
l = x_m[0] ** 2 + x_m[1] ** 2 + x_m[2] ** 2 + u_m[0] ** 2 + u_m[1] ** 2

# Lower bound on the value function (monomial basis)

In [ ]:
def value_function_lb(f, l, X, U, degree):

    # Set up SOS program.
    prog = SosProgram()
    vector_type = type(f[0].vectors()[0])
    basis = vector_type.construct_basis(x, degree)#, odd=False)
    J = prog.add_polynomial(basis)[0]

    # Maximize volume beneath the value function.
    Jint = J.definite_integral(x, - xobj, xobj)
    prog.add_linear_cost(- Jint.to_scalar())

    # S-procedure for the state limits.
    basis = vector_type.construct_basis(x + u, degree // 2)
    Sprocedure = Polynomial({})
    for Xi in X:
        lamxi = prog.add_sos_polynomial(basis)[0] # NewEvenDegreeSosPolynomial
        Sprocedure += lamxi * Xi

    # S-procedure for the input limits.
    for Ui in U:
        lamui = prog.add_sos_polynomial(basis)[0] # NewEvenDegreeSosPolynomial
        Sprocedure += lamui * Ui

    # Bellman inequality.
    Jdot = sum(J.derivative(xi) * f[i] for i, xi in enumerate(x))
    prog.add_sos_constraint(Jdot + l - Sprocedure)

    # Value function nonpositive in the origin.
    prog.add_linear_constraint(J({xi: 0 for xi in x}) <= 0)

    # Solve and retrieve result.
    prog.solve()
    Jlb = prog.substitute_minimizer(J)
    obj = - prog.minimum()
    
    return Jlb, obj

In [ ]:
# Solve for increasing degree.
degrees = np.arange(1, 4) * 2
Jlb = {d: value_function_lb(f, l, X, U, d) for d in degrees}

In [ ]:
# Plot value-function sections.
def plot_value_function(Jlb):
    for d in degrees:
        for i, xi in enumerate(x):
            Jlb_i = Jlb[d][0].substitute({xi: 0})
            xobj_i = np.delete(xobj, i)
            plt.figure()
            x_but_i = [xj for xj in x if xj != xi]
            xlabel = f'${x_but_i[0]}$'
            ylabel = f'${x_but_i[1]}$'
            zlabel = r'$J_{\mathrm{lb}}$'
            title = f'Degree {d}, section ${xi}=0$ (objective {round(Jlb[d][1], 3)})'
            level_plot(Jlb_i, - xobj_i, xobj_i, title=title,
                       xlabel=xlabel, ylabel=ylabel, zlabel=zlabel)
plot_value_function(Jlb)

# Lower bound on the value function (Chebyshev basis)

In [ ]:
# Translate polynomial data in Chebyshev basis.
f_c = [fi.in_chebyshev_basis() for fi in f]
l_c = l.in_chebyshev_basis()
X_c = [Xi.in_chebyshev_basis() for Xi in X]
U_c = [Ui.in_chebyshev_basis() for Ui in U]

# Solve for increasing degree.
Jlb_c = {d: value_function_lb(f_c, l_c, X_c, U_c, d) for d in degrees}

In [ ]:
# Plot value-function sections with Chebyshev basis.
plot_value_function(Jlb_c)